# Evaluation Tracer の構築

EvaluationTracerを構築し、データを取得・分析する準備を行います。

## Jupyter環境の準備

JupyterからSupersetなどの他のコンテナへの操作を行えるようにDockerをインストールしておきます。Jupyter上でdockerコマンドを実行するために/var/run/docker.sockのボリュームマウントが必要ですが、これはコンテナ起動時に使うdocker-compose.ymlに記述済みです。

本Jupyterコンテナ[niicloudoperation/notebook](https://hub.docker.com/r/niicloudoperation/notebook/)は[Dockerfile](https://github.com/NII-cloud-operation/Jupyter-LC_docker/blob/master/Dockerfile)を確認すると分かるようにdebian:jessieベースなので[公式ドキュメント：debianへのインストール方法](https://docs.docker.com/engine/installation/linux/docker-ce/debian/#install-using-the-repository)の手順を用います。

In [ ]:
!sudo apt-get update && sudo apt-get upgrade -y && \
sudo apt-get install -y apt-transport-https ca-certificates curl gnupg2 software-properties-common

In [ ]:
!curl -fsSL https://download.docker.com/linux/debian/gpg | sudo apt-key add -
!sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/debian $(lsb_release -cs) stable"
!sudo apt-get update && sudo apt-get install -y docker-ce

dockerコマンドの動作確認を行います。
psで起動しているコンテナを確認すると、このJupyterの他に、Supersetとその動作に必要なRedis、PostgreSQLのコンテナが起動していて、
合計4つのコンテナが動作中であることが確認できます。

In [ ]:
!sudo docker -v
!sudo docker ps

## Supersetの初期化

Supersetを使うためには、管理ユーザを作成するなどの初期化が必要です。dockerコマンド経由で、Supersetコンテナに、初期化スクリプトを実行させます。この時、引数として、管理ユーザ作成のための情報を渡します。必要なら、usernameやpasswordを修正して、実行してください。

この初期化を行うとSupersetにアクセスしadmin/etpasswordでログインできるようになります。コンテナの動いているサーバの8088番ポートにwebブラウザでアクセスし、確認してください。

Python環境を整えます。

In [ ]:
!sudo $CONDA3_DIR/bin/conda update conda --quiet --yes

In [ ]:
!sudo $CONDA3_DIR/bin/conda update --all --quiet --yes

Supersetのコンテナ名を確認し、Ansible経由で初期化を実施します。

Ansibleって言いたいだけなので、そこはスルーしてください。

playbookは[et-setup.yml](et-setup.yml)にあります。

In [ ]:
import subprocess

ret = subprocess.check_output(["sudo", "docker", "ps", "--filter", "name=superset", "--format", "{{.Names}}"])
superset_container_name = ret.decode('utf-8').strip()

print("superset_container_name: " + superset_container_name)

print(subprocess.check_output(
    ['sudo', 'ansible-playbook', '-i', superset_container_name+',', 'et-setup.yml', '-t', 'init'])
      .decode('utf-8'))

## Superset の Evaluation Tracere 化

EvaluationTracerの中にはメテオラが入っていなければなりません。cssを改変して、メテオラを入れます。

Supersetにアクセスし、webブラウザの開発者機能を使って、stylesheetの適用状況を確認してみたところ、/static/assets/dist/theme.86bf0d5b20edd25ee233.cssの7481行から記述されているbodyセクションに背景画像を設定することができそうだと分かりました。
このpathがSupersetコンテナ内のどこのディレクトリになるのか分かれば、改変できることになります。

このコンテナを作った[Dockerfile](https://github.com/mnagaku/superset/blob/master/Dockerfile)を確認すると、supersetはpipでインストールされていることが分かるので、pipでインストールされたモジュールがどこのディレクトリに置かれるのかが分かれば良いことになります。

グーグル先生に質問しながらコンテナ内を探ってみると、/usr/local/lib/python3.5/dist-packages/superset/static/assets/dist/theme.86bf0d5b20edd25ee233.cssを見つけることができました。

このcssに背景画像を設定する行を追記します。

貼り込む画像は、同じディレクトリの「bg.jpg」を指定しています。もし、別の形式の画像ファイルを貼り込みたい場合は、適切な拡張子を指定するようにしてください。

先ほど取得したSupersetのコンテナ名を指定して、処理を行います。

In [ ]:
import subprocess

print(subprocess.check_output(
    ['sudo', 'ansible-playbook', '-i', superset_container_name+',', 'et-setup.yml', '-t', 'change_css'])
      .decode('utf-8'))

wgetを使って、貼り込む画像をSupersetのコンテナ内にダウンロードします。ダウンロードする画像のURL指定は、お好みに合わせて変更してください。jpg以外の画像形式にしたい場合も、保存先の名前の拡張子を変更してください。

In [ ]:
import subprocess

print(subprocess.check_output(
    ['sudo', 'ansible-playbook', '-i', superset_container_name+',', 'et-setup.yml', '-t', 'get_jpg'])
      .decode('utf-8'))

## Evaluation Tracer ができあがったら

続けて、各種データを取得・グラフ化してみましょう。

[Re:CREATORS](recreators.ipynb)

[NewGame!](newgame1.ipynb)

[NewGame!!](newgame2.ipynb)


In [ ]:
from IPython.display import SVG

SVG(filename='notebooks-filled.svg')